### Parker Validation
- No imputation

In [1]:
options(repr.matrix.max.cols=150, repr.matrix.max.rows=200)

In [2]:
wd <- dirname(dirname(getwd()))
source(paste0(wd,"/mission_control/treasure_map.R"))
source(paste0(wd,"/mission_control/helpers/validation/settings.R"))

library(tidyverse)
library(stringr)

Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.1
Warning message:
“package ‘tibble’ was built under R version 3.6.3”Warning message:
“package ‘tidyr’ was built under R version 3.6.3”Warning message:
“package ‘purrr’ was built under R version 3.6.3”Warning message:
“package ‘dplyr’ was built under R version 3.6.3”Warning message:
“package ‘forcats’ was built under R version 3.6.3”── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──


#### Read data sets

In [3]:
I_DIR <- paste0(E_DIR, "parker_melanoma/")
clinical <- read.csv( paste0(I_DIR, "Clinical/Subjects-CancerCell-MORRISON1-metadata.tsv"), sep = "\t")
somatic <- read.csv( paste0(I_DIR, "WES/WES-CancerCell-MORRISON1-metadata.tsv"), sep = "\t") 
rna <- read.csv( paste0(I_DIR, "RNASeq/data/RNA-CancerCell-MORRISON1-combat_batch_corrected-logcpm-all_samples.tsv"), sep = "\t")
#rna <- read.csv( paste0(I_DIR, "RNASeq/data/RNA-CancerCell-MORRISON1-no_batch_correction-logcpm-all_samples.tsv"), sep = "\t")
rna_link <- read.csv( paste0(I_DIR,"RNASeq/RNA-CancerCell-MORRISON1-metadata.tsv"), sep = "\t") #%>% transmute(sample.id, patient_id = subject.id)

#### 0 - Clean clinical data

In [4]:
binary_response <- function( recist ){
    recist = as.character(recist)
    if( recist %in% c("CR","PR")){
        1
    } else if ( recist %in% c("SD", "PD")){
        0
    } else {
        NA
    }
}

In [5]:
clinical$bor <- unlist(lapply( as.character(clinical$"bor"), binary_response))

In [6]:
clinical_go <- (
    clinical
        %>% filter(! treatment.regimen.name %in% c("CTLA4", "PD1-to-CTLA4"),
                   !cohort %in% c("064", "va") ### stated not pd1 in paper
                  )
        %>% transmute(
          patient_id = subject.id, 
          bor,
          os_days = os,   
          os = NA, 
          os_event = NA, 
          age = subject.age, 
          gender = subject.sex,
          tissue = "skin", 
          tissue_full = sample.tumor.type,
          pretreat = ifelse(previous.treatment == "naive", 0, 1),
          pretreat_comp = NA,
          Study = "PARKER",
          cohort = cohort,
          extra = treatment.regimen.name, 
          extra2 = meddra.disease.preferred.name
        )
)

#### 1 - Somatic data

In [7]:
somatic_go <- (
    somatic
        %>% filter(! treatment.regimen.name %in% c("CTLA4", "PD1-to-CTLA4", "CTLA4-to-PD1"), #, "PD1-to-CTLA4", "CTLA4-to-PD1"
                  grepl("pre",timepoint.id)
                   )
        %>% drop_na(tmb)
        %>% transmute(
            patient_id = subject.id,
            tmb = log(tmb + 1), 
            purity
        )
)

#### 2 - RNA data

- Data to share

In [8]:
share <- data.frame(t(rna %>% column_to_rownames("gene.hgnc.symbol")))
saveRDS( share, paste0(REF_DIR, "rna_parker.Rds") )

- Gene sets

In [9]:
rna_link <- rna_link %>% transmute(sample.id, patient_id = subject.id)

In [10]:
rna_clean <- (
    rownames_to_column(
        data.frame(
            t(column_to_rownames(data.frame(rna %>% filter( gene.hgnc.symbol %in% unlist(gene_sets) )), "gene.hgnc.symbol")
        )
    ), "sample.id")
)

In [11]:
rna_clean$tcell  <- apply(rna_clean %>% select( any_of(gene_sets$clusters$tcell)),1,mean)
rna_clean$prolif <- apply(rna_clean %>% select( any_of(gene_sets$clusters$prolif)),1,mean)
rna_clean$tgfb   <- apply(rna_clean %>% select( any_of(gene_sets$clusters$tgfb)),1,mean)

rna_clean$tcell_cluster5  <- apply(rna_clean %>% select( any_of(gene_sets$clusters5$tcell)),1,mean)
rna_clean$prolif_cluster5 <- apply(rna_clean %>% select( any_of(gene_sets$clusters5$prolif)),1,mean)
rna_clean$tgfb_cluster5   <- apply(rna_clean %>% select( any_of(gene_sets$clusters5$tgfb)),1,mean)

rna_clean$tcell_set  <- apply(rna_clean %>% select( any_of(gene_sets$sets1$tcell)),1,mean)
rna_clean$prolif_set <- apply(rna_clean %>% select( any_of(gene_sets$sets1$prolif)),1,mean)
rna_clean$tgfb_set   <- apply(rna_clean %>% select( any_of(gene_sets$sets1$tgfb)),1,mean)

rna_clean$pdl1   <- apply(rna_clean %>% select( CD274 ),1,mean)

In [12]:
rna_go <- rna_clean %>% select(sample.id, tcell, prolif, tgfb, tcell_cluster5, prolif_cluster5, tgfb_cluster5, tcell_set, prolif_set, tgfb_set, pdl1)

#### 3 - Combined

In [13]:
parker_go <- (
    clinical_go 
        %>% left_join( somatic_go, by , by = "patient_id") 
        %>% left_join( rna_link, by = "patient_id") 
        %>% left_join( rna_go, by = "sample.id")
        %>% filter( !grepl("_on_", sample.id),  !grepl("_ON_", sample.id), !grepl("EDT", sample.id)) 
)

### 4 - Send it!

In [14]:
saveRDS( parker_go, paste0( TMP_DIR, "validation-parker-go.Rds"))